## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Rayagada,IN,19.1667,83.4167,76.68,94,96,2.86,overcast clouds,0.0,0
1,1,Punta Arenas,CL,-53.1500,-70.9167,41.11,93,90,4.61,overcast clouds,0.0,0
2,2,Vila Do Maio,CV,15.1333,-23.2167,79.59,90,99,8.32,overcast clouds,0.0,0
3,3,New Norfolk,AU,-42.7826,147.0587,42.30,77,87,1.99,overcast clouds,0.0,0
4,4,Clyde River,CA,70.4692,-68.5914,44.28,78,97,0.78,overcast clouds,0.0,0


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the min temperature you would like for your trip? "))
max_temp = float(input("What is the max temperature you would like for your trip? "))
rain_bool = input("Do you want it to be raining? (yes/no) ")
snow_bool = input("Do you want it to be snowing? (yes/no) ")

What is the min temperature you would like for your trip? 65
What is the max temperature you would like for your trip? 85
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
if rain_bool == "no" and snow_bool == "no":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (inches)"] == 0) & \
                                        (city_data_df["Snow (inches)"] == 0)]
elif rain_bool == "no" and snow_bool == "yes":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (inches)"] == 0) & \
                                        (city_data_df["Snow (inches)"] >= 0.0)]
elif rain_bool == "yes" and snow_bool == "no":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (inches)"] >= 0.0) & \
                                        (city_data_df["Snow (inches)"] == 0.0)]
else:
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (inches)"] >= 0.0) & \
                                        (city_data_df["Snow (inches)"] >= 0.0)]

In [5]:
# 4a. Determine if there are any empty rows.
clean_df = filtered_cities_df.dropna()

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Rayagada,IN,19.1667,83.4167,76.68,94,96,2.86,overcast clouds,0.0,0
2,2,Vila Do Maio,CV,15.1333,-23.2167,79.59,90,99,8.32,overcast clouds,0.0,0
6,6,Crotone,IT,39.0851,17.1178,77.94,48,0,2.55,clear sky,0.0,0
8,8,Arraial Do Cabo,BR,-22.9661,-42.0278,68.88,82,2,15.52,clear sky,0.0,0
9,9,Anzio,IT,41.4853,12.6179,82.72,64,0,6.04,clear sky,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
676,676,Aragarcas,BR,-15.8975,-52.2508,77.76,29,0,3.13,clear sky,0.0,0
678,678,Cosala,MX,24.4125,-106.6908,78.12,78,41,4.76,scattered clouds,0.0,0
679,679,Barentu,ER,15.1139,37.5928,71.08,85,70,5.19,broken clouds,0.0,0
680,680,Foix,FR,42.8333,1.5833,72.73,95,71,3.60,broken clouds,0.0,0


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rayagada,IN,76.68,overcast clouds,19.1667,83.4167,
2,Vila Do Maio,CV,79.59,overcast clouds,15.1333,-23.2167,
6,Crotone,IT,77.94,clear sky,39.0851,17.1178,
8,Arraial Do Cabo,BR,68.88,clear sky,-22.9661,-42.0278,
9,Anzio,IT,82.72,clear sky,41.4853,12.6179,
14,Norsup,VU,77.25,scattered clouds,-16.0667,167.3833,
17,Albany,US,68.63,overcast clouds,42.6001,-73.9662,
18,Asau,RO,66.96,clear sky,46.4333,26.4000,
20,Rikitea,PF,74.79,clear sky,-23.1203,-134.9692,
29,Butaritari,KI,83.39,few clouds,3.0707,172.7902,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
# 7. Drop the rows where there is no Hotel Name.
fixed_hotel_df = hotel_df.dropna()

In [15]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation.csv"
# 8b. Export the City_Data into a csv
fixed_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in fixed_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = fixed_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
city_locations = hotel_df[["Lat", "Lng"]]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Thats all, folks!